# EDA

## Introduction

- Explore the target variable in `application_{train|test}.csv`. Is the data unbalanced with respect to the target? What would the accuracy be for a simple model consisting in a majority class classifier?

- Explore the relationship between target and predictors, looking for potentially strong predictors that could be included later in a model.
The skimr package in R has some great data exploration tools, and the janitor package has utilities that will simplify data cleaning.

- Explore the scope of missing data in application_{train|test}.csv and come up with possible solutions. Remove rows?  Remove columns?  Impute?

- Be alert to problems in the data.  Do the values make sense? Are there mistaken values that should be cleaned or imputed? (Note that outliers are not necessarily mistakes. Check APM for advice on how to handle outliers for a predictive project.) Are there columns with near-zero or zero variance?

- Will the input data need to be transformed in order to be used in a model? You might answer this question differently for different models. (For example, some models might accept factor variables, others might not.) You can put off answering this question for now,  but you should begin thinking about it. Check APM for advice.

- Join application_{train|test}.csv with transactional data in, for example, bureau.csv or previous_application.csv. This will require aggregating the transactional data to have the same grain as the application data.

- Explore the joined transactional data.  Do some of the added columns show promise in predicting default?


## Import package

In [ ]:
# Install packakges
!pip3 install pandas
!pip3 install numpy


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip3 install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip3 install --upgrade pip


In [ ]:
# import packages
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

## Import dataset

## Data Cleansing

**Handle missing values**
This is one of the most common issues with real-world data. You can handle missing values in a few ways:

**Drop rows with missing values**
Fill in missing values with the mean, median, or mode of the column
Fill in missing values with a constant value (like 0)
The approach you take depends on the nature of the data and your analysis goals.

**Remove duplicates**
Duplicate rows can skew your analysis, so it’s important to identify and remove them. You can use pandas .duplicated() and .drop_duplicates() methods.

**Check for outliers**
Outliers can significantly impact your results, so you’ll want to identify them and either remove them or handle them appropriately. You can use box plots, histograms, and z-scores to find outliers.

**Correct data types**
Ensure all columns have the correct data type (string, integer, float, boolean, etc.). You may need to convert some columns using .astype().

**Fix inconsistent data**
Look for inconsistencies in the data, like spelling variations, different formats, etc., and standardize the data.

**Address class imbalance (if relevant)**
If you have a target variable with class imbalance, you may need to resample the data to get meaningful results.

**Feature engineering**
Derive new features from your existing data that might be useful for your analysis. This is an important step for creating an effective model.

Those are the main steps for cleaning your data as part of the exploratory data analysis process.

## Data Proprecessing

## Data Visualization